In [72]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

## Question 3


In [60]:
img1_gray = cv2.imread('./image1.png', cv2.IMREAD_GRAYSCALE) # READS IN NUMPY ARRAY
# img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
# img1_gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)

img2_gray = cv2.imread('./image2.png', cv2.IMREAD_GRAYSCALE) # READS IN NUMPY ARRAY
# img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
# img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

output = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'output')
if not os.path.isdir(output):
    os.makedirs(output)


### Step 1

In [62]:
def compute_gradient_mag_direction(img, threshold):
    grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)

    # magnitude
    magnitude = cv2.magnitude(grad_x, grad_y)
    magnitude[magnitude < threshold] = 0
    
    # direction
    radians = np.arctan2(grad_y, grad_x)
    radians[radians < 0] += np.pi
    direction = np.degrees(radians)

    return magnitude, direction

img1_gradient_mag, img1_direction = compute_gradient_mag_direction(img1_gray, 0.01)
img2_gradient_mag, img2_direction = compute_gradient_mag_direction(img2_gray, 0.01)


(321, 481)


### Step 2 & 3

In [92]:
def array_cell_grid(size: int, array: np.ndarray) -> np.ndarray:
    height, width = array.shape
    m, n = height // size, width // size
    result = array.copy()
    
    crop_height = height - (m * size)
    crop_width = width - (n * size)

    # Center the grid
    if crop_height > 0:
        if crop_height % 2 == 0:
            crop = int(crop_height / 2)
            result = result[0+crop: -crop, :]
        else:
            result = result[1+crop: -crop,
                            :] if crop_height // 2 > 0 else result[1:, :]

    if crop_width > 0:
        if crop_width % 2 == 0:
            crop = int(crop_width / 2)
            result = result[:, 0+crop: -crop]
        else:
            result = result[:, 1+crop: -
                            crop] if crop_width // 2 > 0 else result[:, 1:]

    return m, n, result


def orientation_histogram(gradient_mag: np.ndarray, gradient_direction: np.ndarray,
                          size: int) -> np.ndarray:

    direction = gradient_direction.copy()
    direction += 15
    direction[direction >= 180] -= 180

    m, n, gm_cropped = array_cell_grid(size, gradient_mag)

    mag_output = np.zeros((m, n, 6))
    occ_output = np.zeros((m, n, 6))

    i, j = 0, 0
    r, c = 0, 0

    for i in range(0, m):
        for j in range(0, n):
            bins_mag = [0, 0, 0, 0, 0, 0]
            bins_occ = [0, 0, 0, 0, 0, 0]
            for r in range(i * size, i * size + size):
                for c in range(j * size, j * size + size):
                    bin_index = int(direction[r, c] // 30)
                    bins_mag[bin_index] += gm_cropped[r, c]
                    bins_occ[bin_index] += 1

            mag_output[i, j] = bins_mag
            occ_output[i, j] = bins_occ
    
    return mag_output, occ_output



# four histograms, two for each image
img1_mag_histogram, img1_occ_histogram = orientation_histogram(
    img1_gradient_mag, img1_direction, 8)
img2_mag_histogram, img2_occ_histogram = orientation_histogram(
    img2_gradient_mag, img2_direction, 8)


#### Quiver Plots


In [67]:
def generate_quiver_values(histogram, bin, size):
    x, y, u, v = [], [], [], []
    m, n, _ = histogram.shape
   
    for i in range(m):
        for j in range(n):
                pad = size // 2
                magnitude = histogram[i, j, bin] * 0.0000005
                rad_angle = np.pi / 6 * bin # angle in center of bin

                # location
                x.append(j * size + pad)
                y.append(i * size + pad)

                # x, y components
                u.append(magnitude * np.sin(rad_angle))
                v.append(magnitude * np.cos(rad_angle))

    return x, y, u, v


##### Image 1 Quiver Plots

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 20))
ax1.imshow(img1_gray, cmap='gray')
ax2.imshow(img1_gray, cmap='gray')
ax1.set_title("Image 1 Magnitude Histogram")
ax2.set_title("Image 1 Occurrence Histogram")

for k in range(6):
    x, y, u, v = generate_quiver_values(img1_mag_histogram, k, 8)
    x, y, u, v = generate_quiver_values(img1_occ_histogram, k, 8)
    ax1.quiver(x, y, u, v, color="red", width=0.0009)
    ax2.quiver(x, y, u, v, color="red", width=0.0009)

plt.savefig(output + '/image1_histograms.png')
plt.show()

##### Image 2 Quiver Plots

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 20))
ax1.imshow(img2_gray, cmap='gray')
ax2.imshow(img2_gray, cmap='gray')
ax1.set_title("Image 2 Magnitude Histogram")
ax2.set_title("Image 2 Orientation Histogram")

for k in range(6):
    x, y, u, v = generate_quiver_values(img1_mag_histogram, k, 8)
    x, y, u, v = generate_quiver_values(img1_occ_histogram, k, 8)
    ax1.quiver(x, y, u, v, color="red", width=0.0009)
    ax2.quiver(x, y, u, v, color="red", width=0.0009)

plt.savefig(os.path.join(output, 'image2_histograms.png'))
plt.show()

### Step 4: Normalization

In [66]:
def normalize(array: np.ndarray):
    e = 0.001
    denominator = np.sqrt(np.sum(np.square(array)) + e)
    return np.divide(array, denominator)

def normalize_histogram(histogram: np.ndarray) -> np.ndarray:
    block_size = 2
    m, n, _ = histogram.shape
    normalized_hist = np.zeros((m - 1, n - 1, 24))

    for i in range(m - 1):
        for j in range(n - 1):
            descriptor = histogram[i:i + block_size, j:j + block_size, :].flatten()
            normalized_hist[i][j] = normalize(descriptor)

    return normalized_hist

normalized_hist_img1 = normalize_histogram(img1_mag_histogram)
normalized_hist_img2 = normalize_histogram(img2_mag_histogram)

np.savetxt(output + '/image1.txt', normalized_hist_img1.reshape(normalized_hist_img1.shape[0], -1), fmt='%1.2f', delimiter=' ', newline='\n')
np.savetxt(output + '/image2.txt', normalized_hist_img2.reshape(normalized_hist_img2.shape[0], -1), fmt='%1.2f', delimiter=' ', newline='\n')

#### Two Phone Images, W/ and W/O Flash

In [78]:
img1 = cv2.imread('./flash.png') # READS IN NUMPY ARRAY
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img1_gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
img1_downsized = cv2.resize(img1_gray, (0,0), fx=0.125, fy=0.125)

img2 = cv2.imread('./no-flash.png') # READS IN NUMPY ARRAY
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
img2_downsized = cv2.resize(img2_gray, (0,0), fx=0.125, fy=0.125)

##### Computing HOG & Visualizing

In [93]:
# Regular Size
img1_gradient_mag, img1_direction = compute_gradient_mag_direction(img1_gray, 0.01)
img2_gradient_mag, img2_direction = compute_gradient_mag_direction(img2_gray, 0.01)

img1_mag_histogram, img1_occ_histogram = orientation_histogram(img1_gradient_mag, img1_direction, 8)
img2_mag_histogram, img2_occ_histogram = orientation_histogram(img2_gradient_mag, img2_direction, 8)

# downsized
img1_downsized_gm, img1_downsized_dir = compute_gradient_mag_direction(img1_downsized, 0.01)
img2_downsized_gm, img2_downsized_dir = compute_gradient_mag_direction(img2_downsized, 0.01)

img1_downsized_mag_histogram, img1_downsized_occ_histogram = orientation_histogram(img1_downsized_gm, img1_downsized_dir, 8)
img2_downsized_mag_histogram, img2_downsized_occ_histogram = orientation_histogram(img2_downsized_gm, img2_downsized_dir, 8)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 20))
ax1.imshow(img1_gray, cmap='gray')
ax2.imshow(img1_gray, cmap='gray')
ax1.set_title("Flash Magnitude Histogram")
ax2.set_title("Flash Orientation Histogram")

for k in range(6):
    x, y, u, v = generate_quiver_values(img1_mag_histogram, k, 8)
    x, y, u, v = generate_quiver_values(img1_occ_histogram, k, 8)
    ax1.quiver(x, y, u, v, color="red", width=0.0009)
    ax2.quiver(x, y, u, v, color="red", width=0.0009)

plt.savefig(output+"/flash_histograms.png")
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 20))
ax1.imshow(img2_gray, cmap='gray')
ax2.imshow(img2_gray, cmap='gray')
ax1.set_title("No Flash Magnitude Histogram")
ax2.set_title("No Flash Orientation Histogram")

for k in range(6):
    x, y, u, v = generate_quiver_values(img2_mag_histogram, k, 8)
    x, y, u, v = generate_quiver_values(img2_occ_histogram, k, 8)
    ax1.quiver(x, y, u, v, color="red", width=0.0009)
    ax2.quiver(x, y, u, v, color="red", width=0.0009)

plt.savefig(output + "/no_flash_histograms.png")
plt.show()

In [97]:
# have to downsize bc text file too large

downsized_normalized_hist_img1 = normalize_histogram(img1_downsized_mag_histogram)
downsized_normalized_hist_img2 = normalize_histogram(img2_downsized_mag_histogram)


np.savetxt(output + '/flash.txt', downsized_normalized_hist_img1.reshape(downsized_normalized_hist_img1.shape[0], -1), fmt='%1.3f', delimiter=' ', newline='\n')
np.savetxt(output + '/no-flash.txt', downsized_normalized_hist_img2.reshape(downsized_normalized_hist_img2.shape[0], -1), fmt='%1.3f', delimiter=' ', newline='\n')

In [33]:
shape = normalized_hist_img1.shape
grad_hist = np.zeros((shape[0], shape[1], 6))

k = 1
for i in range(shape[0]):
    for j in range(shape[1]):
        while k < 6:
            sum = 0 
            for l in range(k, 24, 4):
                sum += normalized_hist_img1[i, j, l-1]
            sum /= 6
            grad_hist[i, j, k - 1] = sum
            k += 1
        

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 20))
ax1.imshow(img1_gray, cmap='gray')
ax2.imshow(img2_gray, cmap='gray')
ax1.set_title("Flash Histogram")
ax2.set_title("No Flash Histogram")

for k in range(6):
    x, y, u, v = generate_quiver_values(grad_hist, k, 8)
    x_1, y_1, u_1, v_1 = generate_quiver_values(grad_hist, k, 8)
    ax1.quiver(x, y, u, v, color="red", width=0.001)
    ax2.quiver(x_1, y_1, u_1, v_1, color="red", width=0.001)

plt.savefig(output + '/flash_mag_histograms.png')
plt.show()

## Question 4

In [35]:
img1 = cv2.imread('./UC_lawn_1.jpg') # READS IN NUMPY ARRAY
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img1_gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)

img2 = cv2.imread('./UC_lawn_2.jpg') # READS IN NUMPY ARRAY
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

### Step 1: Calculate Eigenvalues of M

In [36]:
def eigenval_second_moment_matrix(img: np.ndarray, sigma=1) -> np.ndarray:
    height, width = img.shape
    # get the gradients
    Ix = cv2.Sobel(img, cv2.CV_64F, 1, 0)
    Iy = cv2.Sobel(img, cv2.CV_64F, 0, 1)

    # Get Ix^2, Iy^2, and IxIy
    Ix_2 = np.square(Ix)
    Iy_2 = np.square(Iy)
    IxIy = np.multiply(Ix, Iy)

    # Convolve each one with gaussian filter
    Ix_2 = cv2.GaussianBlur(Ix_2, (5, 5), sigma)
    Iy_2 = cv2.GaussianBlur(Iy_2, (5, 5), sigma)
    IxIy = cv2.GaussianBlur(IxIy, (5, 5), sigma)

    # 3d array, with two slots in last dimension to store lambda_1 & lambda_2
    # for each pixel
    eigenvalues = np.zeros((height, width, 2))
    for i in range(height):
        for j in range(width):
            N = np.array([[Ix_2[i][j], IxIy[i][j]], [IxIy[i][j], Iy_2[i][j]]])
            e_vals = np.linalg.eigvals(N)
            eigenvalues[i][j] = e_vals
            
    return eigenvalues

In [37]:
# Eigen values for each image
I1_e = eigenval_second_moment_matrix(img1_gray)
I2_e = eigenval_second_moment_matrix(img2_gray)

### Step 2: Scatter Plots

### Scatter Plots

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
img1_e1, img1_e2 = I1_e[:, :, 0], I1_e[:, :, 1]
img2_e1, img2_e2 = I2_e[:, :, 0], I2_e[:, :, 1]

ax1.set_xlabel("lambda 1")
ax1.set_ylabel("lambda 2")
ax1.set_title("Image 1 Eigenvalues")
ax1.scatter(img1_e1, img1_e2, s=0.25)

ax2.set_xlabel("lambda 1")
ax2.set_ylabel("lambda 2")
ax2.set_title("Image 2 Eigenvalues")
ax2.scatter(img2_e1, img2_e2, s=0.25)

plt.savefig(output + '/lawn_scatter.png')
plt.show()

### Step 3: Threshold to Detect Corners

In [ ]:
lambda_min_1 = np.minimum(I1_e[:, :, 0], I1_e[:, :, 1])
lambda_min_2 = np.minimum(I2_e[:, :, 0], I2_e[:, :, 1])
threshold = 30000


img1[lambda_min_1 > threshold] = [255, 255, 100]
img2[lambda_min_2 > threshold] = [255, 255, 100]


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.imshow(img1)
ax2.imshow(img2)

plt.savefig(output + '/lawn_corners.png')

### Step 4: Repeated with Larger sigma

#### Step 1

In [40]:
I1_e = eigenval_second_moment_matrix(img1_gray, 12)
I2_e = eigenval_second_moment_matrix(img2_gray, 12)

#### Step 2

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
img1_e1, img1_e2 = I1_e[:, :, 0], I1_e[:, :, 1]
img2_e1, img2_e2 = I2_e[:, :, 0], I2_e[:, :, 1]

ax1.set_xlabel("lambda 1")
ax1.set_ylabel("lambda 2")
ax1.set_title("Image 1 Eigenvalues")
ax1.scatter(img1_e1, img1_e2, s=0.25)

ax2.set_xlabel("lambda 1")
ax2.set_ylabel("lambda 2")
ax2.set_title("Image 2 Eigenvalues")
ax2.scatter(img2_e1, img2_e2, s=0.25)

plt.savefig(output + '/large_sigma_scatter.png')
plt.show()

#### Step 3

In [ ]:
lambda_min_1 = np.minimum(I1_e[:, :, 0], I1_e[:, :, 1])
lambda_min_2 = np.minimum(I2_e[:, :, 0], I2_e[:, :, 1])
threshold = 30000


img1[lambda_min_1 > threshold] = [255, 255, 100]
img2[lambda_min_2 > threshold] = [255, 255, 100]


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.imshow(img1)
ax2.imshow(img2)
plt.savefig(output + '/lawn_corner_larger_sigma.png')